In [1]:
from AutoML_preprocess import cross_valid
import numpy as np
import pandas as pd
import pickle
import os
import datetime

In [2]:
def load_pickle(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

def load_data(dataset_name):
    dataset_path = os.path.join("data", f"{dataset_name}.pkl")
    indexes_path = os.path.join("data", f"{dataset_name}_splits.pkl")
    df = pd.read_pickle(dataset_path)
    indexes = load_pickle(indexes_path)
    return df, indexes

In [3]:
def model_data(datasets, y_index_column):
    modeling_results = {}
    for i, sets in enumerate(datasets):
        print(sets)
        print(48 * "-")
        df, indexes = load_data(sets)
        X = df.drop(df.columns[y_index_column[i]], axis=1)
        y = df[df.columns[y_index_column[i]]]
        try:
            results = cross_valid(X, y, indexes)
        except Exception as e:
            print(e)
            return modeling_results
        modeling_results[sets] = results
    return modeling_results

In [4]:
datasets = ['airlines', 'albert'] # trzeba dodac 'airlines', target na koncu
y_index_column = [-1, 0]

In [5]:
score = model_data(datasets, y_index_column)

airlines
------------------------------------------------
albert
------------------------------------------------


In [6]:
score

{'airlines': {'accuracy': [0.5545523647082816,
   0.5545523647082816,
   0.5545523647082816,
   0.554562646000964,
   0.554562646000964,
   0.554562646000964,
   0.554562646000964,
   0.554562646000964,
   0.554562646000964,
   0.5545441061960028],
  'auc': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]},
 'albert': {'accuracy': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
  'auc': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]}}

In [7]:
s = pd.DataFrame.from_dict(score)
s.to_csv("folds_results_4.csv")
df = s.applymap(lambda x: f"{np.mean(np.array(x))} +- {np.std(np.array(x))}")
df.to_csv("results_4.csv")